# Load dependencies

In [3]:
#load dependencies
from sql_conn import connection #SQL connection details and variables
#import neuralnet as nn #functions to form neural network and calculate accuracy and plot accuracy with history
import preproc as pr #functions to preprocess data 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from sklearn import preprocessing #for one-hot encoding

# SQL Script

In [6]:
s='''
    
    SELECT IC.ICDBlock, 
    SS.StudyID,
    SS.StudyShortName,
    SS.StudyTitle,
    SS.StudyResearchSummary,
    SS.StudyInclusionCriteria
    from 
    (SELECT * 
    FROM [CPMS_BI].dbo.[ODP_Study] WHERE StudyICDCodingRequired='Yes') AS SS 
    LEFT JOIN 
    (SELECT * from [CPMS_BI].[dbo].[ODP_StudyICD] 
    where ICDBlock = 'Other forms of heart disease') 
    AS IC ON SS.StudyID = IC.StudyID
    '''

In [58]:
study_df = pd.read_sql(s, connection)

C:\Users\busaji\AppData\Local\Temp\ipykernel_25916\3074574563.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  study_df = pd.read_sql(s, connection)


In [56]:
study_df.dtypes

ICDBlock                  object
StudyID                    int64
StudyShortName            object
StudyTitle                object
StudyResearchSummary      object
StudyInclusionCriteria    object
dtype: object

In [59]:
#pre-processing
study_df2 = study_df.fillna("not available")

#convert to float
#study_df['StudyResearchSummary'] = study_df['StudyResearchSummary'].astype("string")

In [47]:
study_df2

,ICDBlock,StudyID,StudyShortName,StudyTitle,StudyResearchSummary,StudyInclusionCriteria
0,0,646,RT01,A Randomised Trial of High Dose Therapy in Loc...,To compare standard 64 Gy dose of conformal ra...,1) Histologically confirmed cancer of the pros...
1,0,651,PR07,A Randomised trial of Hormone Therapy plus Rad...,To evaluate any possible benefit from the addi...,Histologically confirmed Prostate cancer withi...
2,0,760,ProtecT,The ProtecT Trial - Evaluating the effectivene...,The overall aim is to evaluate the effectivene...,- inclusion for randomisation: men with locali...
3,0,791,Intercontinental,INTERmittent or CONTINuos ENdocrine Therapy Af...,A phase III randomised trial comparing intermi...,Patients with adenocarcinoma of the prostate w...
4,0,798,EORTC 30985,Intermittent androgen deprivation in patients ...,0,not specified
...,...,...,...,...,...,...
19508,0,63960,Reducing NGT feeding under restraint & co-desi...,Learning from a positive deviant and exploring...,Naso-gastric tube (NGT) feeding under physical...,Inclusion Criteria Phase One Young People ...
19509,0,63982,Helping Hand,Hand measurements in EMG clinics,The diagnosis of motor neurone disease (MND) r...,Healthy Volunteer • No known neurological pr...
19510,0,64063,Evaluation of a patient decision aid for ulcer...,Evaluation of a patient decision aid for ulcer...,Ulcerative Colitis (UC) is a lifelong Inflamma...,"Patients 1. Participant of any gender, aged o..."
19511,0,64087,"UK MyREMEDY Clinical Trial, Version 1",MyDiaMate for Remission of Elevated Diabetes D...,One in three persons affected by type 1 diabet...,All participants must be 18 years of age or ol...


In [51]:
#workaround - but 'reload' is not defined
import sys
reload(sys)
sys.setdefaultencoding('utf8')

NameError: name 'reload' is not defined

1. Convert to string

In [60]:
#study_df2['StudyResearchSummary'] = study_df2['StudyResearchSummary'].encode('utf-8')

for index, row in study_df2.iterrows():
    row['StudyResearchSummary'] = row['StudyResearchSummary'].encode('utf-8')




# TF-IDF Vectorizer

In [61]:

corpus = study_df2['StudyResearchSummary']
vectorizer = TfidfVectorizer(lowercase = True, encoding = "utf-8", decode_error="ignore", stop_words='english', analyzer='word', ngram_range=(1,2))
X = vectorizer.fit_transform(corpus)
vectorizer.get_feature_names_out()
print(X.shape)

(19513, 956142)


In [62]:
X.head()

AttributeError: 'csr_matrix' object has no attribute 'head'

# K-Means Clustering

In [63]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=20, random_state=0, n_init="auto").fit(X)


In [64]:
feat_nms = kmeans.get_feature_names_out()

In [65]:
feat_nms

array(['kmeans0', 'kmeans1', 'kmeans2', 'kmeans3', 'kmeans4', 'kmeans5',
       'kmeans6', 'kmeans7', 'kmeans8', 'kmeans9', 'kmeans10', 'kmeans11',
       'kmeans12', 'kmeans13', 'kmeans14', 'kmeans15', 'kmeans16',
       'kmeans17', 'kmeans18', 'kmeans19'], dtype=object)

In [67]:
kmeans.

KMeans(n_clusters=20, random_state=0)

# LDA

In [66]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.datasets import make_multilabel_classification

lda = LatentDirichletAllocation(n_components=5,
    random_state=0)
lda.fit(X) #fit X


LatentDirichletAllocation(n_components=5, random_state=0)

In [68]:
#transform X
X_lda =lda.transform(X)



In [71]:
X_lda.shape

(19513, 5)

In [ ]:
#get output as pandas
X_lda.set_output(*,transform="pandas")